<a href="https://colab.research.google.com/github/lagom-QB/M11/blob/master/EMNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# EMNIST

EMNIST is an extention of MNIST dataset. It has 47 classes (handwritten digits and leters), some of the letters are represented as two classes (upper and lower case) and some (ex. o, s) has only one class associated with them.

In [0]:
from torchvision import datasets, transforms
import torch

train_dataset = datasets.EMNIST('/data', train=True, download=True,
                                transform=transforms.Compose([
                                    transforms.ToTensor(),
                                    transforms.Normalize((0.1307,), (0.3081,))
                                ]),
                                split="balanced")

test_dataset = datasets.EMNIST('../data', train=False, download=True,
                                transform=transforms.Compose([
                                    transforms.ToTensor(),
                                    transforms.Normalize((0.1307,), (0.3081,))
                                ]),
                                split="balanced")

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=True)

#1

Find correspondence between classess and letters/digits

In [0]:
letters = train_dataset.classes_split_dict['letters']
digits  = train_dataset.classes_split_dict['digits']
byclass = train_dataset.classes_split_dict['byclass']  # if not why not
balanced = train_dataset.classes_split_dict['balanced']

In [3]:
len(balanced)

47

#2

Build and train a dense classifier for EMNIST dataset. Try to achieve the highest accuracy you can.\
Explain, what experiments have you perform.\
What optimizers have you tested?\
How network/optimizer parameters were chosen.

In [0]:
import torch.nn as nn
from torch.nn.modules import loss

class Net(nn.Module):
    
    def __init__(self):
        
        super(Net, self).__init__()

        self.layer1 = nn.Sequential(
                        nn.Conv2d(1, 32, kernel_size=5, stride=1, padding=2),
                        nn.ReLU(),
                        nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
                        nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2),
                        nn.ReLU(),
                        nn.MaxPool2d(kernel_size=2, stride=2))
        self.drop_out = nn.Dropout()
        self.fc1 = nn.Linear(7 * 7 * 64, 1000)
        self.fc2 = nn.Linear(1000, 64)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.drop_out(out)
        out = self.fc1(out)
        out = self.fc2(out)
        return out

In [0]:
learning_rate = 0.001 # Small enough
num_epochs = 25 #If not why not
num_classes = len(balanced)
batch_size = 100

In [0]:
model = Net()

device    = torch.device("cuda")
# device    = torch.device("cpu")

model = model.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()

optimizer_1 = torch.optim.Adam(
                    model.parameters(), 
                    lr=learning_rate)  

optimizer = torch.optim.Adagrad(
                    model.parameters(),
                    lr = learning_rate, 
                    lr_decay = 0.01)
scheduler = torch.optim.lr_scheduler.StepLR(
                                            optimizer,
                                            0.5)

In [0]:
def test_ass(model, test_loader, loss_function, device = device):
    model.eval()
    
    test_loss = 0
    correct = 0
    
    with torch.no_grad():
        for data, target in test_loader:
            data = data.to(device)
            target = target.to(device)
    
            output = model(data)
    
            test_loss += loss_function(output, target).sum().item()
    
            pred = output.argmax(dim=1, keepdim=True)
    
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [0]:
def train_ass(model, train_loader, optimizer, loss_function, epoch , device = device):
    model.train()

    for batch_idx, (data, target) in enumerate(train_loader):
        data = data.to(device)
        target = target.to(device)

        optimizer.zero_grad()

        output = model(data)
        loss = loss_function(output, target)
        
        loss.backward()
        optimizer.step()
        
        if batch_idx % 500 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [13]:
for epoch in range(20):
    train_ass(model, train_loader, optimizer, loss.CrossEntropyLoss(), epoch)    
    test_ass(model, test_loader, loss.CrossEntropyLoss(), device = device)

Train Epoch: 0 [0/112800 (0%)]	Loss: 4.196727
Train Epoch: 0 [32000/112800 (28%)]	Loss: 0.858494
Train Epoch: 0 [64000/112800 (57%)]	Loss: 1.318224
Train Epoch: 0 [96000/112800 (85%)]	Loss: 0.919112

Test set: Average loss: 0.0157, Accuracy: 13380/18800 (71%)

Train Epoch: 1 [0/112800 (0%)]	Loss: 1.190591
Train Epoch: 1 [32000/112800 (28%)]	Loss: 1.029113
Train Epoch: 1 [64000/112800 (57%)]	Loss: 0.981955
Train Epoch: 1 [96000/112800 (85%)]	Loss: 0.948933

Test set: Average loss: 0.0153, Accuracy: 13496/18800 (72%)

Train Epoch: 2 [0/112800 (0%)]	Loss: 0.714079
Train Epoch: 2 [32000/112800 (28%)]	Loss: 1.223826
Train Epoch: 2 [64000/112800 (57%)]	Loss: 1.179669
Train Epoch: 2 [96000/112800 (85%)]	Loss: 0.796930

Test set: Average loss: 0.0152, Accuracy: 13545/18800 (72%)

Train Epoch: 3 [0/112800 (0%)]	Loss: 1.114707
Train Epoch: 3 [32000/112800 (28%)]	Loss: 1.099747
Train Epoch: 3 [64000/112800 (57%)]	Loss: 0.945126
Train Epoch: 3 [96000/112800 (85%)]	Loss: 0.917236

Test set: Average

Adam, lr (0.01) >> Accuracy of 88%\
Adagrad, lr (0.01) >> Accuracy of 73%\
 .

#3

Train the same classifier on MNIST dataset (you will have to replace the last dense layer of your model)

*MNIST*

In [15]:
from torchvision import datasets, transforms
import torch

train_dataset_mnist = datasets.MNIST(
                                '/data', 
                                train = True, 
                                download = True,
                                transform=transforms.Compose([
                                    transforms.ToTensor(),
                                    transforms.Normalize((0.1307,), (0.3081,))
                                ]))
test_dataset_mnist = datasets.MNIST(
                                '../data', 
                                train = False, 
                                download = True,
                                transform = transforms.Compose([
                                    transforms.ToTensor(),
                                    transforms.Normalize((0.1307,), (0.3081,))
                                ]))
train_loader_mnist = torch.utils.data.DataLoader(
                                        train_dataset_mnist, 
                                        batch_size=64, 
                                        shuffle=True)

test_loader_mnist = torch.utils.data.DataLoader(
                                        test_dataset_mnist, 
                                        batch_size=64, 
                                        shuffle=True)

Extracting /data/MNIST/raw/train-images-idx3-ubyte.gz to /data/MNIST/raw


Extracting /data/MNIST/raw/train-labels-idx1-ubyte.gz to /data/MNIST/raw


Extracting /data/MNIST/raw/t10k-images-idx3-ubyte.gz to /data/MNIST/raw


Extracting /data/MNIST/raw/t10k-labels-idx1-ubyte.gz to /data/MNIST/raw
Processing...
Done!


/pytorch/torch/csrc/utils/tensor_numpy.cpp:141: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program.


In [0]:
class MNISTNet(nn.Module):
    
    def __init__(self):
        
        super(MNISTNet, self).__init__()

        self.layer1 = nn.Sequential(
                        nn.Conv2d(1, 32, kernel_size=5, stride=1, padding=2),
                        nn.ReLU(),
                        nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
                        nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2),
                        nn.ReLU(),
                        nn.MaxPool2d(kernel_size=2, stride=2))
        self.drop_out = nn.Dropout()
        self.fc1 = nn.Linear(7 * 7 * 64, 1000)
        self.fc2 = nn.Linear(1000, 10) # changed from self.fc2 = nn.Linear(1000, 64)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.drop_out(out)
        out = self.fc1(out)
        out = self.fc2(out)
        return out

model_mnist = MNISTNet()

In [22]:
model_mnist = model.to(device)

optimizer_mnist_1 = torch.optim.Adam(
                    model_mnist.parameters(), 
                    lr=learning_rate)  

optimizer_mnist = torch.optim.Adagrad(
                    model_mnist.parameters(),
                    lr = learning_rate, 
                    lr_decay = 0.01)

scheduler = torch.optim.lr_scheduler.StepLR(
                                            optimizer_mnist,
                                            0.5)
for epoch in range(20):
    train_ass(model_mnist, train_loader, optimizer_mnist, loss.CrossEntropyLoss(), epoch)    
    test_ass(model_mnist, test_loader, loss.CrossEntropyLoss(), device = device)

Train Epoch: 0 [0/112800 (0%)]	Loss: 1.192002
Train Epoch: 0 [32000/112800 (28%)]	Loss: 0.659615
Train Epoch: 0 [64000/112800 (57%)]	Loss: 0.799771
Train Epoch: 0 [96000/112800 (85%)]	Loss: 0.807819

Test set: Average loss: 0.0112, Accuracy: 14765/18800 (79%)

Train Epoch: 1 [0/112800 (0%)]	Loss: 0.368608
Train Epoch: 1 [32000/112800 (28%)]	Loss: 0.908488
Train Epoch: 1 [64000/112800 (57%)]	Loss: 0.814740
Train Epoch: 1 [96000/112800 (85%)]	Loss: 0.608257

Test set: Average loss: 0.0111, Accuracy: 14800/18800 (79%)

Train Epoch: 2 [0/112800 (0%)]	Loss: 0.595504
Train Epoch: 2 [32000/112800 (28%)]	Loss: 0.581754
Train Epoch: 2 [64000/112800 (57%)]	Loss: 0.724877
Train Epoch: 2 [96000/112800 (85%)]	Loss: 0.577215

Test set: Average loss: 0.0110, Accuracy: 14826/18800 (79%)

Train Epoch: 3 [0/112800 (0%)]	Loss: 0.714643
Train Epoch: 3 [32000/112800 (28%)]	Loss: 0.685011
Train Epoch: 3 [64000/112800 (57%)]	Loss: 0.489106
Train Epoch: 3 [96000/112800 (85%)]	Loss: 0.674667

Test set: Average

In [23]:
for epoch in range(20):
    train_ass(model_mnist, train_loader, optimizer_mnist_1, loss.CrossEntropyLoss(), epoch)    
    test_ass(model_mnist, test_loader, loss.CrossEntropyLoss(), device = device)

Train Epoch: 0 [0/112800 (0%)]	Loss: 0.660424
Train Epoch: 0 [32000/112800 (28%)]	Loss: 0.616922
Train Epoch: 0 [64000/112800 (57%)]	Loss: 0.634622
Train Epoch: 0 [96000/112800 (85%)]	Loss: 0.349676

Test set: Average loss: 0.0067, Accuracy: 16078/18800 (86%)

Train Epoch: 1 [0/112800 (0%)]	Loss: 0.571385
Train Epoch: 1 [32000/112800 (28%)]	Loss: 1.116911
Train Epoch: 1 [64000/112800 (57%)]	Loss: 0.442031
Train Epoch: 1 [96000/112800 (85%)]	Loss: 0.330028

Test set: Average loss: 0.0062, Accuracy: 16312/18800 (87%)

Train Epoch: 2 [0/112800 (0%)]	Loss: 0.457874
Train Epoch: 2 [32000/112800 (28%)]	Loss: 0.294371
Train Epoch: 2 [64000/112800 (57%)]	Loss: 0.481055
Train Epoch: 2 [96000/112800 (85%)]	Loss: 0.848505

Test set: Average loss: 0.0060, Accuracy: 16339/18800 (87%)

Train Epoch: 3 [0/112800 (0%)]	Loss: 0.323048
Train Epoch: 3 [32000/112800 (28%)]	Loss: 0.291554
Train Epoch: 3 [64000/112800 (57%)]	Loss: 0.777489
Train Epoch: 3 [96000/112800 (85%)]	Loss: 0.684747

Test set: Average

Adagrad(79%)\
Adam(88%)


#4

Use EMNIST classifier to classify MNIST test dataset.\

* To do this you'll need to restrict last layer to the labels which correspond to digits and choose the largest value 

(\
    torch tensors can be indexed with lists, like this:\
    --> torch.tensor(
        [
            [1,2],
            [3,4], 
            [5, 6]
        ])[[0, 1]]\
).\
\
Which of the models performs better?

In [0]:
def test_ass_2(model, test_loader, loss_function, device = device):
    model.eval()
    
    test_loss = 0
    correct = 0
    
    with torch.no_grad():
        for data, target in test_loader:
            data = data.to(device)
            target = target.to(device)
    
            output = model(data)
    
            test_loss += loss_function(output[:,0, 10], target).sum().item()
    
            pred = output[:,0, 10].argmax(dim=1, keepdim=True)
    
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [0]:
def train_ass_2(model, train_loader, optimizer, loss_function, epoch , device = device):
    model.train()

    for batch_idx, (data, target) in enumerate(train_loader):
        data = data.to(device)
        target = target.to(device)

        optimizer.zero_grad()

        output = model(data)
        loss = loss_function(output[:,0, 10], target)
        
        loss.backward()
        optimizer.step()
        
        if batch_idx % 500 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [28]:
for epoch in range(20):
    train_ass_2(model_mnist, train_loader, optimizer_mnist_1, loss.CrossEntropyLoss(), epoch)    
    test_ass_2(model_mnist, test_loader, loss.CrossEntropyLoss(), device = device)

IndexError: ignored

EMNIST is an extention of MNIST dataset.
It has 47 classes (handwritten digits and leters), some of the letters are represented as two classes (upper and lower case) and some (ex. o, s) has only one class associated with them.

1. Find correspondence between classess and letters/digits.
2. Build and train a dense classifier for EMNIST dataset. Try to achieve the highest accuracy you can. Explain, what experiments have you perform. What optimizers have you tested? How network/optimizer parameters were chosen.
3. Train the same classifier on MNIST dataset (you will have to replace the last dense layer of your model).
4. Use EMNIST classifier to classify MNIST test dataset. To do this you'll need to restrict last layer to the labels which correspond to digits and choose the largest value (torch tensors can be indexed with lists, like this: `torch.tensor([[1,2],[3,4], [5, 6]])[[0, 1]]`). Which of the models performs better?
5. Present a short (2-3 pages long) report on this experiment. What approaches have you tried? How can you explain the results you got? Include tensorboard plots in the experiment report.
